### Notebook contains accuracy metrics for asian american-white groups across all emotions and individually as well.

In [1]:
import torch
from emonet import EmoNet
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
import pandas as pd
random.seed(42)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmoNet(n_expression=8)
model.load_state_dict(torch.load("D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\\cfd_finetuned_emonet_100_epochs.pth"))
model.to(device)
model.eval()

In [3]:
device

device(type='cuda')

In [4]:
#defining image transform
cfd_transform=transforms.Compose([ transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])])

In [5]:
emotion_map = {'N': 0, 'A': 1, 'F': 2, 'HC': 3, 'HO': 3}

In [6]:
def load_images(path):
    images_list = []
    label_list = []
    filenames = [f for f in os.listdir(path) if f.endswith('.jpg')]
    
    for file in filenames:
        img_path = os.path.join(path, file)
        last_char = file[-5]
        if last_char=='C' or last_char=='O':
            last_char='H'+last_char
        label = emotion_map[last_char]
        image = Image.open(img_path).convert("RGB")
        image = cfd_transform(image)
        images_list.append(image)
        label_list.append(label)

    images_tensor = torch.stack(images_list) if images_list else torch.empty(0)
    labels_tensor = torch.tensor(label_list, dtype=torch.long) if label_list else torch.empty(0, dtype=torch.long)

    return images_tensor, labels_tensor

### loading asian-american and white datasets

In [8]:
path_nam="D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\Male\Asian american\\Neutral"
path_naf="D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\Female\Asian american\\Neutral"


nam_list,nam_labels=load_images(path_nam)
naf_list,naf_labels=load_images(path_naf)


#we've loaded corresponding images, now stack everything to create one whole tensor for asian american people.

asian_american_images = torch.cat([
    nam_list,naf_list
], dim=0)

asian_american_labels=torch.cat([
nam_labels,naf_labels
])

print("Asian american image tensor shape:", asian_american_images.shape)

Asian american image tensor shape: torch.Size([109, 3, 256, 256])


In [9]:
path_nwm="D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\Male\White\\Neutral"
path_nwf="D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\Female\White\\Neutral"

nwm_list,nwm_labels=load_images(path_nwm)
nwf_list,nwf_labels=load_images(path_nwf)



white_images = torch.cat([
    nwm_list,nwf_list
], dim=0)

white_labels=torch.cat([
nwm_labels,nwf_labels
])

print("white image tensor shape:", white_images.shape)

white image tensor shape: torch.Size([183, 3, 256, 256])


### loading functions for evaluation

In [10]:
#function to extract relevant emotion logits, we only need 4 instead of 8
def extract_relevant_logits(output):
    logits=output["expression"]
    relevant_logits = torch.stack([
        logits[:, 0],  # N
        logits[:, 6],  # A
        logits[:, 4],  # F
        logits[:, 1],  # (HappyClosedMouth + HappyOpenMouth)
    ], dim=1) 

    return relevant_logits

In [11]:
def predict_emonet(image_path, model):
    image = Image.open(image_path).convert("RGB")
    image = cfd_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        predicted_label = torch.argmax(extract_relevant_logits(output), dim=1).item()
    
    return predicted_label

In [12]:
def calculate_accuracy(image_tensor, label_tensor, model):
    correct = 0
    total = image_tensor.shape[0] 
    with torch.no_grad():
        for i in range(total):
            image = image_tensor[i].unsqueeze(0).to(device)
            true_label = label_tensor[i].item() 
            output = model(image)
            pred_label = torch.argmax(extract_relevant_logits(output), dim=1).item()

            if pred_label == true_label:
                correct += 1

    return correct / total * 100 if total > 0 else 0

### all asian-american/white neutral images accuracy calculation

In [14]:
asian_american_neutral_images = torch.cat([nam_list, naf_list], dim=0)
asian_american_neutral_labels = torch.cat([nam_labels, naf_labels])

white_neutral_images = torch.cat([nwm_list, nwf_list], dim=0)
white_neutral_labels = torch.cat([nwm_labels, nwf_labels])

print("Asian american neutral image tensor shape:", asian_american_neutral_images.shape)
print("White neutral image tensor shape:", white_neutral_images.shape)

Asian american neutral image tensor shape: torch.Size([109, 3, 256, 256])
White neutral image tensor shape: torch.Size([183, 3, 256, 256])


In [15]:
asian_american_neutral_accuracy = calculate_accuracy(asian_american_neutral_images, asian_american_neutral_labels, model)
white_neutral_accuracy = calculate_accuracy(white_neutral_images, white_neutral_labels, model)

print(f"Accuracy asian american neutral imgs: {asian_american_neutral_accuracy:.2f}%")
print(f"Accuracy white neutral imgs: {white_neutral_accuracy:.2f}%")

Accuracy asian american neutral imgs: 90.83%
Accuracy white neutral imgs: 93.99%


### accuracy for neutral male aam-w images

In [16]:
asian_american_male_neutral_accuracy = calculate_accuracy(nam_list, nam_labels, model)
white_male_neutral_accuracy = calculate_accuracy(nwm_list, nwm_labels, model)

print(f"Accuracy asian american male neutral imgs: {asian_american_male_neutral_accuracy:.2f}%")
print(f"Accuracy white male neutral imgs: {white_male_neutral_accuracy:.2f}%")

Accuracy asian american male neutral imgs: 94.23%
Accuracy white male neutral imgs: 96.77%


### accuracy for neutral female aam-w images

In [17]:
asian_american_female_neutral_accuracy = calculate_accuracy(naf_list, naf_labels, model)
white_female_neutral_accuracy = calculate_accuracy(nwf_list, nwf_labels, model)

print(f"Accuracy asian american female neutral imgs: {asian_american_female_neutral_accuracy:.2f}%")
print(f"Accuracy white female neutral imgs: {white_female_neutral_accuracy:.2f}%")

Accuracy asian american female neutral imgs: 87.72%
Accuracy white female neutral imgs: 91.11%


### save to csv

In [18]:
accuracy_list=[asian_american_neutral_accuracy,white_neutral_accuracy,asian_american_male_neutral_accuracy,white_male_neutral_accuracy,asian_american_female_neutral_accuracy,white_female_neutral_accuracy]
col_list = [
    "asian_american_neutral_accuracy", "white_neutral_accuracy", "asian_american_male_neutral_accuracy", "white_male_neutral_accuracy",
    "asian_american_female_neutral_accuracy", "white_female_neutral_accuracy"
]


In [19]:
rounded_accuracy = [round(value, 3) for value in accuracy_list]

In [20]:
df=pd.DataFrame(rounded_accuracy,col_list,columns=['accuracy(%)'])
df.head()

,accuracy(%)
asian_american_neutral_accuracy,90.826
white_neutral_accuracy,93.989
asian_american_male_neutral_accuracy,94.231
white_male_neutral_accuracy,96.774
asian_american_female_neutral_accuracy,87.719


In [21]:
df.to_csv('metrics_aam-w_onlyneutral.csv')

### FAIRNESS METRICS PRE-SUPPRESSION

In [22]:
from torch.utils.data import DataLoader, TensorDataset
def get_preds_in_batches_reduced(model, images, batch_size=32, device="cuda"):
    dataset = TensorDataset(images)
    loader = DataLoader(dataset, batch_size=batch_size)
    preds = []

    model.eval()
    with torch.no_grad():
        for (batch,) in loader:
            batch = batch.to(device)
            output = model(batch)
            logits = extract_relevant_logits(output)
            pred = logits.argmax(dim=1).cpu()
            preds.append(pred)

    return torch.cat(preds)

In [23]:
def compute_group_metrics(preds, labels, target_class):
    # True Positive Rate (TPR): correctly predicted as class y / all actual class y
    # False Positive Rate (FPR): predicted as class y but actual not y / all actual not y
    true_positive = ((preds == target_class) & (labels == target_class)).sum().item()
    false_positive = ((preds == target_class) & (labels != target_class)).sum().item()
    actual_positive = (labels == target_class).sum().item()
    actual_negative = (labels != target_class).sum().item()

    tpr = true_positive / actual_positive if actual_positive > 0 else 0
    fpr = false_positive / actual_negative if actual_negative > 0 else 0

    return tpr, fpr

In [24]:
def compute_equalized_odds_gap(model, asian_american_images, asian_american_labels, white_images, white_labels, target_class, device="cuda"):
    preds_b = get_preds_in_batches_reduced(model, asian_american_images, device=device)
    preds_w = get_preds_in_batches_reduced(model, white_images, device=device)

    tpr_b, fpr_b = compute_group_metrics(preds_b, asian_american_labels, target_class)
    tpr_w, fpr_w = compute_group_metrics(preds_w, white_labels, target_class)

    gap = abs(tpr_b - tpr_w) + abs(fpr_b - fpr_w)
    return gap, tpr_b,tpr_w,fpr_b,fpr_w

In [27]:
target_class = 0 #for neutral, which is index 0 in our modified emotion set
gap1,tpr_aam1,tpr_w1,fpr_aam1,fpr_w1 = compute_equalized_odds_gap(model, asian_american_images, asian_american_labels, white_images, white_labels, target_class)
print(f"Equalized Odds Gap (Neutral): {gap1:.3f}")

Equalized Odds Gap (Neutral): 0.032


In [28]:
print(f"TPR Asian american N: {tpr_aam1:.3f}, TPR White N: {tpr_w1:.3f}")
print(f"FPR Asian american N: {fpr_aam1:.3f}, FPR White N: {fpr_w1:.3f}")

TPR Asian american N: 0.908, TPR White N: 0.940
FPR Asian american N: 0.000, FPR White N: 0.000


as observed above, the TPR for aam-N and white neutral images is almost similar and FPR is 0 for both these classes. so no bias can be indicated.